注意就是用Scikit-multilearn的iterative_train_test_split()函数，它可以迭代地创建训练/测试分割以实现平衡标签。需要进行增加多一个维度


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
model_ckpt = "miguelvictor/python-gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
def mean_pooling(model_output, attention_mask):
	# Extract the token embeddings
	token_embeddings = model_output[0]
	# Compute the attention mask
	input_mask_expanded = (attention_mask .unsqueeze(-1) .expand(token_embeddings.size()) .float())
	# Sum the embeddings, but ignore masked tokens
	sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
	sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
	# Return the average as a single vector
	return sum_embeddings / sum_mask
def embed_text(examples):
	inputs = tokenizer(examples["text"], padding=True, truncation=True, max_length=128, return_tensors="pt")
	with torch.no_grad():
		model_output = model(**inputs)
	pooled_embeds = mean_pooling(model_output, inputs["attention_mask"])
	return {"embedding": pooled_embeds.cpu().numpy()}
